In [1]:
# Load amazon books data

from datasets.amazonbook import Amazonbook

amazonbooks_data_config = {
    "name": "amazonbook",
    "rewrite": False,
}

amazonbooks_data = Amazonbook.from_config(amazonbooks_data_config)

2024-04-29 10:45:34,792 : [1/3] DATASET : Loading processed dataset datasets/data/amazonbook/dataset.parquet.


In [2]:
amazon_split_config = {
    "seed": 42,
    "val_target_proportion": 0.0,
}

(amazon_train, amazon_val, amazon_test), amazon_split_time = amazonbooks_data.create_splits(amazon_split_config)

2024-04-29 10:46:25,468 : [1/3] DATASET : Dataframe lengths | train_df: 2380730, val_df: 2380730, test_df: 2984108
2024-04-29 10:47:11,637 : [1/3] DATASET : Removing users [50736, 52234, 41589, 13647] from test inputs.
2024-04-29 10:47:12,271 : [1/3] DATASET : Splits information:
2024-04-29 10:47:12,272 : [1/3] DATASET : Train split info | n_users = 52643, n_items = 91599, n_ratings = 2380730, sparsity = 99.95%
2024-04-29 10:47:12,273 : [1/3] DATASET : Validation split info | n_users = 52643, n_items = 91599, n_ratings = 2380730, sparsity = 99.95%
2024-04-29 10:47:12,274 : [1/3] DATASET : Test split info | n_users = 52639, n_items = 91599, n_ratings = 2380661, sparsity = 99.95%
2024-04-29 10:47:12,276 : [1/3] DATASET : Execution of create_splits took at 97.194 seconds.


In [3]:
# Create dictionary of item occurences for novelty metric
training_csr_matrix = amazon_train.get_csr_matrix()
item_occurrences = training_csr_matrix.sum(axis=0)
item_ids = item_occurrences.nonzero()[1]
# Get the occurrences as a numpy array
item_occurrences = item_occurrences.A1
    
# Create a dictionary of item IDs and their occurrences
item_occurrences_dict = dict(zip(item_ids, item_occurrences))

In [4]:
from models.randomrec import RANDOM_REC

# No need to train, just predict
ran_pred = RANDOM_REC(amazon_test.data, 20)

In [7]:
import recmetrics

amazon_book_novelty, amazon_book_novelty_topn = recmetrics.novelty(ran_pred, item_occurrences_dict, len(amazon_test.data['user_id'].unique().tolist()), 20)

amazon_book_novelty

11.579565591315276